In [1]:
# 1. 導入必要的套件
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain_community.utilities import SerpAPIWrapper
from langchain.tools import Tool
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.memory import ConversationTokenBufferMemory
# 2. 設置環境
import os
from dotenv import load_dotenv
load_dotenv()

# 3. 初始化 LLM
llm = ChatOpenAI(model_name="gpt-4-turbo-preview", temperature=0.1)

# 4. 設置搜尋工具
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="""
        用於搜尋資訊的工具。
        """
    ),
]

# 5. 設置記憶存儲
memory = ConversationTokenBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="output",
    max_token_limit=2000,  # 設定記憶體大小限制
    llm=llm # 使用 LLM 來編碼和解碼記憶體
)

# 6. 創建整合的 prompt 模板
prefix = """你是一個專業的投資顧問助手。使用以下工具來回答人類的問題。請使用繁體中文回答。

請注意：
1. 回答問題時要參考對話歷史中的相關資訊
2. 如果問題提到"根據前面的資訊"，要特別注意使用之前對話中提到的數據
3. 給出投資建議時，要考慮以下面向：
   - 營收與獲利能力
   - 產業發展前景
   - 市場競爭態勢
   - 公司治理情況
   - 股價合理性評估
4. 回答要具體且有數據支持
5. 提醒投資人注意風險"""

format_instructions = """回答時請遵循以下格式：

Question: 輸入的問題
Thought: 你應該思考要做什麼
Action: {tool_names} 中的一個工具名稱
Action Input: 工具的輸入
Observation: 工具的結果
... (這個 Thought/Action/Action Input/Observation 可以重複)
Thought: 我現在知道最終答案
Final Answer: 用繁體中文回答原始問題"""

suffix = """先前對話歷史：
{chat_history}

Question: {input}
{agent_scratchpad}"""

# 7. 創建 prompt 和 agent
prompt = ZeroShotAgent.create_prompt(
    tools=tools,
    prefix=prefix,
    suffix=suffix,
    format_instructions=format_instructions,
    input_variables=["input", "chat_history", "agent_scratchpad"]
)

llm_chain = LLMChain(llm=llm, prompt=prompt)
agent = ZeroShotAgent(
    llm_chain=llm_chain,
    tools=tools,
    verbose=True
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory,
    max_iterations=3,  # 限制最大迭代次數
    early_stopping_method="generate"  # 使用生成式早停
)

# 8. 測試對話
def chat_with_agent(question):
    try:
        response = agent_executor.invoke({
            "input": question
        })
        
        # 檢查回答是否包含必要的元素
        answer = response.get("output", "")
        if not answer or len(answer) < 50:  # 確保回答有一定的長度
            return "抱歉，無法提供完整的回答，請重新提問或提供更多資訊。"
            
        # 如果是投資建議，確保包含風險提醒
        if "投資" in question or "買進" in question or "賣出" in question:
            if "風險" not in answer:
                answer += "\n\n請注意：以上建議僅供參考，投資有風險，入市須謹慎。"
                
        return answer
        
    except Exception as e:
        error_msg = f"發生錯誤：{str(e)}"
        print(f"Error details: {e.__class__.__name__}")  # 輸出詳細錯誤資訊
        return error_msg

C:\Users\pochenkuo\AppData\Local\Temp\ipykernel_14236\1877190532.py:32: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationTokenBufferMemory(
C:\Users\pochenkuo\AppData\Local\Temp\ipykernel_14236\1877190532.py:81: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)
C:\Users\pochenkuo\AppData\Local\Temp\ipykernel_14236\1877190532.py:82: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/l

In [2]:
# 9. 執行測試
question1 = "鈊象電子2024的營收狀況如何？"
print("問題1:", question1)
print("AI回答:", chat_with_agent(question1))

question2 = "以投資的角度來看，前面的資訊是否足夠，若不足夠，請補充"
print("\n問題2:", question2)
print("AI回答:", chat_with_agent(question2))

question3 = "根據前面的資訊，你覺得這家公司值得投資嗎？"
print("\n問題3:", question3)
print("AI回答:", chat_with_agent(question3))

問題1: 鈊象電子2024的營收狀況如何？


> Entering new AgentExecutor chain...
Thought: 為了回答這個問題，我需要查找關於鈊象電子2024年的營收預測或者是相關分析師的報告。
Action: Search
Action Input: 鈊象電子 2024 營收預測
Observation: ['鈊象12月營收站上17億大關，在2024年最後一個月創下年營收和月營收的歷史新高。 12月營收年增29.1%，累積去年營收185.12億，年增30.6%，再次創下歷史新高。', '法人認為，隨著農曆年旺季將至，估計鈊象今年業績仍有望持續走升，加上海外線上博弈執照將陸續到手，有助整體營收規模成長。 展望未來，鈊象目前正積極申請英國 ...', '台灣遊戲開發商鈊象（3293），11/6日公布2024年10月自結合併營收，為16.8億元，相較上個月15.81億元，月增6.23%，相對去年同期12.31億元，年增36.48%，月營收也 ...', '鈊象2024年第四季營收50.61億元，亦創單季新高，較前一年度成長33.5％。 累計2024年全年度鈊象電子的營收為185.12億元，年增30.59％，亦為歷史新高。 整 ...', '三大法人上周買超358張。 從基本面來看，鈊象電子2024年前11月已繳出亮眼成績單，前11月累計營收為168億元，年增30.75％。 ... 預測，不必然代表投資之績效。', '鈊象(3293-TW)今天公告2024年10月營收為新台幣16.80億元，年增率36.48%，月增率6.24%。 今年1-10月累計營收為151.31億元，累計年增率30.25%。 最新價為 ...', '所有投資鈊象的人，對鈊象的成長都會有一個『預期』，比如預期每一年都可以成長5%，假如我們能把這個『預期』給算出來，就能判斷他們是不是太小看鈊象了，如果 ...', '... 收入來源。2023年，美國授權收入佔整體營收的15%。 整體成長預期. 展望2024年，法人平均預估鈊象2024年營收年增率為20.42%，這是根據Refinitiv的統計 ...', '法人預估，鈊象2024年稅前盈餘有機會挑戰百億元門檻，全年有機會賺近三個股本。而唯一的變數在於匯率。由於鈊象電子目前有56％的營收來源均為海外授權 ...', '6.報